In [1]:
import datasets

dataset = datasets.load_dataset("philschmid/markdown-documentation-transformers")

dataset

df = dataset["train"].to_pandas()

df.head()

/home/ubuntu/1xa100-2/llama-stack/envs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,title,url,markdown,html,crawlDate
0,🤗 Transformers,https://huggingface.co/docs/transformers/v4.34...,# 🤗 Transformers\n\nState-of-the-art Machine L...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:32.994Z
1,Quick tour,https://huggingface.co/docs/transformers/v4.34...,# Quick tour\n\nGet up and running with 🤗 Tran...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.381Z
2,Installation,https://huggingface.co/docs/transformers/v4.34...,# Installation\n\nInstall 🤗 Transformers for w...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:34.498Z
3,Pipelines for inference,https://huggingface.co/docs/transformers/v4.34...,# Pipelines for inference\n\nThe [pipeline()](...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.128Z
4,Load pretrained instances with an AutoClass,https://huggingface.co/docs/transformers/v4.34...,# Load pretrained instances with an AutoClass\...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:35.231Z


In [2]:
df = df.rename(columns={'html': 'context'})
# All 300 documents takes about an 1.5 hours to ingest
# So let's sample 100 for now
df = df.sample(n=100).reset_index(drop=True)
df.to_parquet("data/transformers.parquet")
df.head()

,title,url,markdown,context,crawlDate
0,PoolFormer,https://huggingface.co/docs/transformers/v4.34...,# PoolFormer\n\n## Overview\n\nThe PoolFormer ...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:32:52.509Z
1,Object detection,https://huggingface.co/docs/transformers/v4.34...,# Object detection\n\nObject detection is the ...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:33:45.839Z
2,Utilities for `FeatureExtractors`,https://huggingface.co/docs/transformers/v4.34...,# Utilities for \`FeatureExtractors\`\n\nThis ...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:31:15.859Z
3,Perplexity of fixed-length models,https://huggingface.co/docs/transformers/v4.34...,# Perplexity of fixed-length models\n\nPerplex...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:30:55.946Z
4,Sample usage,https://huggingface.co/docs/transformers/v4.34...,# Sample usage\n\n## UMT5\n\n[![Models](https:...,"<!DOCTYPE html><html class=""""><head>\n\t\t<met...",2023-10-05T13:33:19.915Z


In [3]:
from counter import get_and_increment_counter
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{get_and_increment_counter()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_54'

In [4]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]
documents[:5]

[{'document_id': '0',
  'content': '<!DOCTYPE html><html class=""><head>\n\t\t<meta charset="utf-8">\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">\n\t\t<meta name="description" content="We’re on a journey to advance and democratize artificial intelligence through open source and open science.">\n\t\t<meta property="fb:app_id" content="1321688464574422">\n\t\t<meta name="twitter:card" content="summary_large_image">\n\t\t<meta name="twitter:site" content="@huggingface">\n\t\t<meta property="og:title" content="PoolFormer">\n\t\t<meta property="og:type" content="website">\n\t\t<meta property="og:url" content="https://huggingface.co/docs/transformers/v4.34.0/en/model_doc/poolformer">\n\t\t<meta property="og:image" content="https://huggingface.co/front/thumbnails/docs/transformers.png">\n\n\t\t<link rel="stylesheet" href="/front/build/kube-b0520c1/style.css">\n\n\t\t<link rel="preconnect" href="https://fonts.gstatic.com">\n\t\t<link href="https

In [5]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

  0%|                                                                                   | 0/100 [00:00<?, ?it/s]

 27%|███████████████████▉                                                      | 27/100 [06:34<34:43, 28.54s/it]